# Nearest neighbor for spine injury classification

In this homework notebook we use **nearest neighbor classification** to classify back injuries for patients in a hospital, based on measurements of the shape and orientation of their pelvis and spine.

The data set contains information from **310** patients. For each patient, there are: six measurements (the x i.e. features) and a label (the y). The label has **3** possible values, `’NO’` (normal), `’DH’` (herniated disk), or `’SL’` (spondilolysthesis). 

Credits: Edx Machine Learning Fundamentals

# 1. Setup notebook

In [1]:
import numpy as np

Load the data set and divide the data into a training set of 248 patients and a separate test set of 62 patients. The following arrays are created:

* **`trainx`** : The training data's features, one point per row.
* **`trainy`** : The training data's labels.
* **`testx`** : The test data's features, one point per row.
* **`testy`** : The test data's labels.

We will use the training set (`trainx` and `trainy`), with nearest neighbor classification, to predict labels for the test data (`testx`). We will then compare these predictions with the correct labels, `testy`.

Notice that we code the three labels as `0. = ’NO’, 1. = ’DH’, 2. = ’SL’`.

In [4]:
# Load data set and code labels as 0 = ’NO’, 1 = ’DH’, 2 = ’SL’
labels = [b'NO', b'DH', b'SL']
data = np.loadtxt('../Data/NN_Spine/column_3C.dat', converters={6: lambda s: labels.index(s)} )

# Separate features from labels
x = data[:,0:6]
y = data[:,6]

# Divide into training and test set
training_indices = list(range(0,20)) + list(range(40,188)) + list(range(230,310))
test_indices = list(range(20,40)) + list(range(188,230))

trainx = x[training_indices,:]
trainy = y[training_indices]
testx = x[test_indices,:]
testy = y[test_indices]

## 2. Nearest neighbor classification with L2 (*Euclidean*) distance

A brute forces nearest neighbor implementation based on Euclidean distance between a test feature and entire training data set

In [42]:
def get_l_dist(x,y, p=2):
    '''
    computes P normed distance between two arrays
    '''
    
    return (np.sum(abs(x-y)**p))**(1/p)

def NN_Euclidean(trainx, trainy, testx):
    
    '''
    A naive algorithm to find the nearest neighbor without any sorting 
    '''
    
    if len(testx.shape)> 1: # Recursive call
        return np.array(list(map(lambda test_item: NN_Euclidean(trainx, trainy, test_item), testx)))
        
    distances = [get_l_dist(trainx_instance, testx) for trainx_instance in trainx]
    
    return trainy[np.argmin(distances)]

In [43]:
testy_L2 = NN_Euclidean(trainx, trainy, testx)
## Compute the accuracy
accuracy = np.equal(testy_L2, testy)
accuracy = float(np.sum(accuracy))/len(testy)

print("Accuracy of nearest neighbor classifier (Euclidean): %{:.2f}".format(accuracy*100))

Accuracy of nearest neighbor classifier: %79.03


# 3. Nearest neighbor classification with L1 (Manhattan) distance

We now compute nearest neighbors using the L1 distance (sometimes called *Manhattan Distance*).

<font color="magenta">**For you to do:**</font> Write a function, **NN_L1**, which again takes as input the arrays `trainx`, `trainy`, and `testx`, and predicts labels for the test points using 1-nearest neighbor classification. For **NN_L1**, the L1 distance metric should be used. As before, the predicted labels should be returned in a `numpy` array with one entry per test point.

Notice that **NN_L1** and **NN_L2** may well produce different predictions on the test set.

In [46]:
def NN_Manhattan(trainx, trainy, testx):
    
    '''
    A naive algorithm to find the nearest neighbor without any sorting 
    '''
    
    if len(testx.shape)> 1: # Recursive call
        return np.array(list(map(lambda test_item: NN_Manhattan(trainx, trainy, test_item), testx)))
        
    distances = [get_l_dist(trainx_instance, testx, p=1) for trainx_instance in trainx]
    
    return trainy[np.argmin(distances)]

In [47]:
testy_L1 = NN_Manhattan(trainx, trainy, testx)
## Compute the accuracy
accuracy = np.equal(testy_L1, testy)
accuracy = float(np.sum(accuracy))/len(testy)

print("Accuracy of nearest neighbor classifier (Manhattan): %{:.2f}".format(accuracy*100))

Accuracy of nearest neighbor classifier (Manhattan): %77.42


# 4. Confusion matrix

In order to have a more in depth comparison between the two distance functions, we can use the <font color="blue">*confusion matrix*</font> that is often use to evaluate a classifier.

We will now look a bit more deeply into the specific types of errors made by nearest neighbor classification, by constructing the .

|         | NO           | DH  | SL |
| ------------- |:-------------:| -----:|-----:|
| NO     |       |       |       |               |
| DH     |       |       |       |               |
| SL     |       |       |       |               |

Since there are three labels, the confusion matrix is a 3x3 matrix whose rows correspond to the true label and whose columns correspond to the predicted label. For example, the entry at row DH, column SL, contains the number of test points whose correct label was DH but which were classified as SL.

In [64]:
def confusion_matrix(testy,testy_fit):
    dict_label ={0: 'NO', 1: 'DH', 2: 'SL'} 
    n_label = len(dict_label)
    output = np.zeros((n_label,n_label))
    
    if len(testy) != len(testy_fit):
        raise ValueError('The two data sets must have the same length')
        
    for predict, true in zip(testy_fit, testy):
        output[int(true), int(predict)] +=1
    
    return output

In [67]:
L1_cm = confusion_matrix(testy, testy_L1)
L2_cm = confusion_matrix(testy, testy_L2)

print( 'Confusion matrix nearest neighbor classifier (Manhattan):\n', L1_neo )

print( 'Confusion matrix nearest neighbor classifier (Euclidean):\n', L2_neo )

Confusion matrix nearest neighbor classifier (Manhattan):
 [[16.  2.  2.]
 [10. 10.  0.]
 [ 0.  0. 22.]]
Confusion matrix nearest neighbor classifier (Euclidean):
 [[17.  1.  2.]
 [10. 10.  0.]
 [ 0.  0. 22.]]


# 5. Some Observations from the results

*Note down the answers to these, since you will need to enter them as part of this week's assignment.*

* DH was **most frequently** misclassified by the L1-based nearest neighbor classifier?
* SL was **never** misclassified by the L2-based nearest neighbor classifier?
* Only one test point had a different prediction between the two classification schemes (based on L1 and L2 distance)